In [1]:
import keras

Using TensorFlow backend.


In [2]:
!pip install np_utils

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
data = pd.read_csv('Churn_Modelling.csv')
X = data.iloc[:, 3:13]
y = data.iloc[:, -1]

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from collections import defaultdict

possDummyVals = defaultdict(set)

        

def addDummies(df, cols_for_dummies):
    for col in cols_for_dummies:

        df_col = pd.get_dummies(df[col], prefix = col, drop_first = True)
        possDummyVals[col] = set(df_col.columns)
        print('now possDummyVal for '+col+' : '+str(possDummyVals[col]))
        print('num of dummies: '+str(len(df_col.columns)))
        df = pd.concat([df, df_col], axis = 1)
        print("added dummy for "+col)
        df.drop([col], axis = 1, inplace = True)
        print('num of cols in df: '+str(len(df.columns)))
    return df

def preprocess(df):
    le = LabelEncoder()    
    df['Gender'] = le.fit_transform(df['Gender'])
    cols_for_dummies = ['Geography', 'NumOfProducts', 'Tenure']
    if len(df)==1:
        return encodeSingle(df, cols_for_dummies)
    return addDummies(df, cols_for_dummies)

In [5]:
def encodeSingle(df, cols_for_dummies):
    for col in cols_for_dummies:
        val_col = df[col][0]
        print(col+"_"+str(val_col))
        print('set '+ col+"_"+str(val_col) + 'to 1')
        print('num of cols: '+str(len(df.columns)))
        possible_vals = [val for val in possDummyVals[col] if val != val_col]
        print('other vals: '+str(possible_vals))
        for cat_val in possible_vals:
            if cat_val == str(col+"_"+str(val_col)):
                df[cat_val] = 1
            else:
                df[cat_val] = 0
        print('set other '+str(len(possible_vals)) + 'to 0')
        print('num of cols: '+str(len(df.columns)))
        df.drop([col], axis = 1, inplace = True)
        print('encoded: '+col+ ' now num of cols: '+str(len(df.columns)))
    return df

In [6]:
X = preprocess(X)

now possDummyVal for Geography : {'Geography_Spain', 'Geography_Germany'}
num of dummies: 2
added dummy for Geography
num of cols in df: 11
now possDummyVal for NumOfProducts : {'NumOfProducts_3', 'NumOfProducts_4', 'NumOfProducts_2'}
num of dummies: 3
added dummy for NumOfProducts
num of cols in df: 13
now possDummyVal for Tenure : {'Tenure_4', 'Tenure_1', 'Tenure_7', 'Tenure_10', 'Tenure_5', 'Tenure_8', 'Tenure_6', 'Tenure_2', 'Tenure_3', 'Tenure_9'}
num of dummies: 10
added dummy for Tenure
num of cols in df: 22


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2)

In [8]:
#feature scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
num_cols = ['CreditScore', 'Balance', 'EstimatedSalary']

In [9]:
X_train[num_cols] = sc.fit_transform(X_train[num_cols])
X_val[num_cols] = sc.transform(X_val[num_cols])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [10]:
y_train_temp = y_train.copy()
y_val_temp = y_val.copy()
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)


Making the ANN

In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

callbacks = [keras.callbacks.EarlyStopping(monitor='loss', #early stopping metric should really be val_loss if using fit but for cv it might take into consideration validation loss/acc by default, read up on this
                              min_delta=0,
                              patience=10,
                              verbose=1, mode='auto')]

from sklearn.utils import compute_class_weight
class_weights = compute_class_weight('balanced', [0, 1], y_train_temp)

In [16]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(output_dim = 12, init = 'uniform', activation = 'relu',input_dim = 22))
    classifier.add(Dense(output_dim = 12, init = 'uniform', activation = 'relu'))
    classifier.add(Dense(output_dim = 12, init = 'uniform', activation = 'relu'))
    classifier.add(Dense(output_dim = 2, init = 'uniform', activation = 'sigmoid'))
    adam = Adam(lr = 1e-2)
    classifier.compile(optimizer = adam, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

In [17]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 20, epochs = 100)
accuracies = cross_val_score(estimator= classifier, X = X_train, y = y_train, cv = 5, n_jobs = 1, fit_params={'callbacks': callbacks, 'class_weight': class_weights})

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=12, activation="relu", input_dim=22, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=12, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=12, kernel_initializer="uniform")`
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=2, kernel_initializer="uniform")`
  


Epoch 1/100
6400/6400 [==============================] - 2s 302us/step - loss: 0.4548 - acc: 0.7936
Epoch 2/100
6400/6400 [==============================] - 1s 167us/step - loss: 0.4105 - acc: 0.8183
Epoch 3/100
6400/6400 [==============================] - 1s 165us/step - loss: 0.4038 - acc: 0.8238
Epoch 4/100
6400/6400 [==============================] - 1s 162us/step - loss: 0.3907 - acc: 0.8380
Epoch 5/100
6400/6400 [==============================] - 1s 167us/step - loss: 0.3952 - acc: 0.8309
Epoch 6/100
6400/6400 [==============================] - 1s 164us/step - loss: 0.3911 - acc: 0.8348
Epoch 7/100
6400/6400 [==============================] - 1s 167us/step - loss: 0.3896 - acc: 0.8353
Epoch 8/100
6400/6400 [==============================] - 1s 165us/step - loss: 0.3926 - acc: 0.8330
Epoch 9/100
6400/6400 [==============================] - 1s 167us/step - loss: 0.3906 - acc: 0.8344
Epoch 10/100
6400/6400 [==============================] - 1s 165us/step - loss: 0.3825 - acc: 0.8383

In [18]:
mean_acc = accuracies.mean()
std_dev = accuracies.std()

In [19]:
mean_acc

0.8456874990463257

In [20]:
std_dev

0.009965565320929626

In [ ]:

classifier.fit(X_train, y_train, batch_size = 20, epochs = 100, callbacks = callbacks, validation_split = 0.2, class_weight = class_weights)

In [ ]:

classifier.evaluate(X_val, y_val)

In [ ]:
y_pred = classifier.predict_classes(X_val)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_val_temp, y_pred)
cm

In [ ]:
y_pred_prob = classifier.predict(X_val)
y_pred1 = (y_pred_prob[:, 1] > 0.5)
cm1 = confusion_matrix(y_val_temp, y_pred1)
cm1

Predicting a single instance

In [ ]:
rows_list = []
row_dict = {'Geography': 'France', 'CreditScore': 600, 'Gender': 'Male', 'Age': 40, 'Tenure': 3, 'Balance': 60000, 'NumOfProducts': 2, 'HasCrCard': 1, 'IsActiveMember': 1, 'EstimatedSalary': 50000}
rows_list.append(row_dict)
test_df = pd.DataFrame(rows_list)

In [ ]:
test_df = preprocess(test_df)
test_df[num_cols] = sc.transform(test_df[num_cols])

In [ ]:
pred_class = classifier.predict_classes(test_df)
pred_class